In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# define the possible values for genre
genres = ["action", "comedy", "drama", "horror", "romance"]

# generate 1000 random ids
ids = np.arange(1, 1001)
# generate 1000 random filenames
filenames = np.array([f"file_{id}.txt" for id in ids])
# generate a 1000 x 1024 array of random numbers between 0 and 255
x = np.random.randint(0, 256, size=(1000, 1024))
# choose 1000 random genres from the list
genres = np.random.choice(genres, size=1000)

# create a dataframe from the arrays
df = pd.DataFrame(np.column_stack([ids, filenames, x, genres]))
# rename the columns
df.columns = ["id", "filename"] + [f"x{i}" for i in range(1, 1025)] + ["genre"]

# write the dataframe to a csv file
df.to_csv("records.csv", index=False)